# **Black Wednesday**

### **1. Import and clean Data**

In [139]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, wilcoxon
import statsmodels.formula.api as smf
import statsmodels.api as sm
from lets_plot import *
LetsPlot.setup_html()

## **2. Define the theme for the visualisations**

In [140]:
def consistent_theming():
    return theme(
        # Clean, professional background
        panel_background=element_rect(fill='white', color='white'),
        
        # Subtle grid lines - solid for a cleaner professional look
        panel_grid_major=element_line(color='#E6E6E6', size=0.4, linetype='solid'),
        panel_grid_minor=element_line(color='#F5F5F5', size=0.2, linetype='solid'),
        
        # Professional typography with clear hierarchy
        axis_text=element_text(size=11, color='#333333'),
        axis_title=element_text(size=13, color='#333333', face='bold'),
        plot_title=element_text(size=16, color='#333333', face='bold', hjust=0),
        
        # Clean legend
        legend_background=element_rect(fill='none'),
        legend_title=element_text(size=12, color='#333333', face='bold'),
        legend_text=element_text(size=10, color='#333333'),
        
        # Additional professional elements
        axis_line=element_line(color='#333333', size=0.5)
    )


## **3. Import necessary data and clean it**

In [141]:
# Initialise an empty list to store any plots that will be created
plots = []

# Read the Excel file, skipping the first row, then rename certain columns
# to more meaningful names, and finally drop the redundant 'Unnamed: 0' column.
UK_data = pd.read_excel("../data/UK.xlsx", skiprows=1).rename(columns={
    "Unnamed: 1": "Month",
    "Economic Activity, Industrial Production, Index": "Industrial_Production",
    "Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate": "National_Exchange_Rate",
    "Exchange Rates, Real Effective Exchange Rate based on Consumer Price Index, Index": "Real_Effective_Exchange_Rate",
    "Prices, Consumer Price Index, All items, Index": "CPI"
}).drop("Unnamed: 0", axis=1)

def clean_and_new_vars(df):
    # Copy the DataFrame to avoid modifying the original dataset
    df_clean = df.copy()
    
    # Remove rows where 'Month' is just a 4-digit year (e.g., '2022'),
    # as these are likely aggregate or non-useful entries
    df_clean = df_clean[~df_clean['Month'].str.match(r'^\d{4}$')]
    
    # Convert the textual month-year data into a proper datetime format
    df_clean.loc[:, 'Date'] = pd.to_datetime(df_clean['Month'], format='%b %Y')
    
    # Calculate month-on-month percentage growth for the real effective exchange rate
    df_clean.loc[:, 'Real_Exchange_Rate_Growth'] = (
        df_clean['Real_Effective_Exchange_Rate'].pct_change() * 100
    )
    
    # Calculate 12-month (year-on-year) inflation rate from the CPI
    df_clean.loc[:, "CPI_Inflation"] = (
        df_clean["CPI"].pct_change(periods=12) * 100
    )
    
    # Calculate 12-month (year-on-year) growth rate for industrial production
    df_clean.loc[:, "Industrial_Production_Growth"] = (
        df_clean["Industrial_Production"].pct_change(periods=12) * 100
    )
    
    # Drop the original 'Month' column, remove any rows with missing data,
    # and reset the index for the cleaned DataFrame
    return (
        df_clean.drop(["Month"], axis=1)
                .dropna()
                .reset_index(drop=True)
    )

# Clean the UK_data DataFrame and create additional growth/inflation columns
UK_data = clean_and_new_vars(UK_data)


In [142]:
UK_data

,Industrial_Production,National_Exchange_Rate,Real_Effective_Exchange_Rate,CPI,Date,Real_Exchange_Rate_Growth,CPI_Inflation,Industrial_Production_Growth
0,63.430733,0.415835,152.957121,35.412040,1981-01-01,3.029780,13.045236,-11.855670
1,69.829448,0.435708,152.778371,35.731304,1981-02-01,-0.116863,12.459836,-8.505468
2,70.571328,0.448625,149.318729,36.267653,1981-03-01,-2.264484,12.609049,-8.313253
3,63.801673,0.459186,149.934490,37.314818,1981-04-01,0.412380,12.039896,-5.364512
4,62.596118,0.478525,150.196746,37.557448,1981-05-01,0.174914,11.740108,-6.380028
...,...,...,...,...,...,...,...,...
247,102.398828,0.695950,126.686198,83.003514,2001-08-01,-1.253315,2.046385,3.901895
248,104.925838,0.683720,127.910878,83.225448,2001-09-01,0.966704,1.626016,1.812367
249,113.935177,0.689020,127.493038,83.114481,2001-10-01,-0.326665,1.490515,6.468172
250,113.935177,0.696190,127.831377,83.114481,2001-11-01,0.265378,1.216216,3.184080


### **4. Is there evidence that the UK joining the ERM and leaving after Black Wednesday led to changes in the volatility of the monthly growth in the real exchange rate?**

In [143]:
date_joined_ERM = pd.to_datetime("Oct 1990")
date_left_ERM = pd.to_datetime("Sept 1992")

In [144]:
# Plot the monthly percentage change in the Real Effective Exchange Rate (growth),
# adding vertical lines at the dates the UK joined and left the ERM.
Real_Exchange_Rate_Growth_Plot = (
    ggplot(UK_data, aes(x='Date', y='Real_Exchange_Rate_Growth')) +
    ggtitle("Volatility decreased when they joined") +
    geom_line(color='blue', size=1.0) +  
    geom_vline(xintercept=date_joined_ERM, color='red', size=0.5) +
    geom_vline(xintercept=date_left_ERM,   color='red', size=0.5) +
    xlab('Date') +
    ylab('Real Effective Exchange Rate growth') +
    consistent_theming() + 
    ggsize(1000,600)
)

# Append this plot to the 'plots' list for later use (e.g. displaying or saving)
plots.append((Real_Exchange_Rate_Growth_Plot, "Real_Exchange_Rate_Growth_Plot"))

# Calculate the rolling standard deviation over a 12-month window of the Real Effective Exchange Rate.
# Standard deviation is a common measure of volatility in financial/economic time series.
UK_data['Volatility'] = UK_data['Real_Effective_Exchange_Rate'].rolling(window=12).std()

# Plot the 12-month rolling volatility of the Real Effective Exchange Rate,
# again marking the ERM entry and exit dates with vertical red lines.
Real_Exchange_Rate_Growth_Volatility_Plot = (
    ggplot(UK_data, aes(x='Date', y='Volatility')) +
    ggtitle("and increased sharply when they left") +
    geom_line(color='blue', size=1.0) +
    geom_vline(xintercept=date_joined_ERM, color='red', size=0.5) +
    geom_vline(xintercept=date_left_ERM, color='red', size=0.5) +
    xlab('Date') +
    ylab('Volatility (12-month Rolling STD)') +
    consistent_theming() + 
    ggsize(1000,600)
)

# Append this volatility plot to the 'plots' list for easy reference
plots.append((Real_Exchange_Rate_Growth_Volatility_Plot, "Real_Exchange_Rate_Growth_Volatility_Plot"))

# Display both plots side by side in a grid, adjusting columns and widths for readability
gggrid([Real_Exchange_Rate_Growth_Plot, Real_Exchange_Rate_Growth_Volatility_Plot], 
       ncol=2, widths=[800, 800], fit=True)


### **5. Are there any corresponding changes in the volatility of either the difference in inflation between the UK and Germany or the growth in industrial production?**

In [145]:
# Create a new DataFrame to hold both UK's and Germany's inflation data
German_UK_inflation = pd.DataFrame()

# Populate this new DataFrame with the dates and UK's CPI inflation values
German_UK_inflation["Date"] = UK_data["Date"]
German_UK_inflation["UK_CPI_Inflation"] = UK_data["CPI_Inflation"]

# Read Germany's CPI data from an Excel file, skip the first row, and rename key columns
German_CPI = pd.read_excel("../data/GermanyERM.xlsx", skiprows=1).rename(columns={
    "Unnamed: 1": "Month",
    "Economic Activity, Industrial Production, Index": "Industrial_Production",
    "Prices, Consumer Price Index, All items, Index": "CPI"
}).drop("Unnamed: 0", axis=1)

# Exclude rows where 'Month' is just a 4-digit year, as these rows are not needed
German_CPI = German_CPI[~German_CPI['Month'].str.match(r'^\d{4}$')]

# Convert the 'Month' column into a proper datetime format and then drop the column
German_CPI['Date'] = pd.to_datetime(German_CPI['Month'], format='%b %Y')
German_CPI = German_CPI.drop("Month", axis=1)

# Calculate year-on-year inflation rate for Germany (12-month percentage change) 
German_CPI["German_CPI_Inflation"] = German_CPI["CPI"].pct_change(periods=12) * 100

# Remove the raw CPI column after calculating inflation
German_CPI = German_CPI.drop("CPI", axis=1)

# Merge Germany's inflation data with the existing UK inflation data using their common dates
German_UK_inflation = pd.merge(German_UK_inflation, German_CPI, on="Date")

# Drop any rows with missing values to keep the dataset consistent
German_UK_inflation = German_UK_inflation.dropna()

# Calculate the inflation difference between the UK and Germany, which can reveal relative
# movements in price levels between the two countries
German_UK_inflation["Difference"] = (
    German_UK_inflation["UK_CPI_Inflation"] - German_UK_inflation["German_CPI_Inflation"]
)

In [146]:
# Plot the UK CPI inflation trend over time, highlighting the UK's entry and exit from the ERM.
UK_Inflation_Plot = (
    ggplot(German_UK_inflation, aes(x='Date', y='UK_CPI_Inflation')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('UK CPI Inflation') +
    xlab('Date') +
    ylab('Inflation (%)') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
# Store the plot object in 'plots' with a descriptive name
plots.append((UK_Inflation_Plot, "UK_Inflation_Plot"))

# Plot Germany's CPI inflation trend similarly, again marking ERM entry and exit.
German_Inflation_Plot = (
    ggplot(German_UK_inflation, aes(x='Date', y='German_CPI_Inflation')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('UK inflation relative to Germany fell dramatically during the ERM period.') +
    xlab('Date') +
    ylab('Inflation (%)') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
# Store this plot object in 'plots' for future reference
plots.append((German_Inflation_Plot, "German_Inflation_Plot"))

# Visualise the difference in CPI inflation between the UK and Germany (UK minus Germany).
# This highlights periods where inflation rates diverge or converge between the two.
Difference_In_Inflation_Plot = (
    ggplot(German_UK_inflation, aes(x='Date', y='Difference')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('Difference in Inflation (UK - Germany)') +
    xlab('Date') +
    ylab('Inflation Difference (%)') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
# Append to our list of plots for a consolidated view
plots.append((Difference_In_Inflation_Plot, "Difference_In_Inflation_Plot"))

# Display the three plots (difference, UK inflation, and German inflation) in a grid layout.
gggrid([Difference_In_Inflation_Plot, UK_Inflation_Plot, German_Inflation_Plot])


In [147]:
# Visualise the UK's year-on-year industrial production growth trend over time,
# with dashed red lines indicating the dates the UK joined and left the ERM.
UK_Industrial_Growth_Plot = (
    ggplot(UK_data, aes(x='Date', y='Industrial_Production_Growth')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('UK Industrial Production Growth was predominantly negative during the ERM period.') +
    xlab('Date') +
    ylab('Industrial Production Growth (%)') +
    consistent_theming() + 
    ggsize(1000,600)
)

# Add this plot to the list of plots so it can be displayed or saved later
plots.append((UK_Industrial_Growth_Plot,"UK_Industrial_Growth_Plot"))

# Display the plot for immediate viewing
UK_Industrial_Growth_Plot

In [148]:
# Calculate the 12-month rolling standard deviation of the inflation difference (UK - Germany),
# which measures how much the gap between UK and German inflation rates fluctuates over time.
German_UK_inflation["Inflation_Difference_Volatility"] = (
    German_UK_inflation["Difference"]
    .rolling(window=12)
    .std()
)

# Plot the volatility of the inflation difference and add dashed red lines to show
# when the UK joined and left the ERM, providing visual context for these events.
Inflation_Difference_Volatility_Plot = (
    ggplot(German_UK_inflation, aes(x='Date', y='Inflation_Difference_Volatility')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('Volatility of the inflation difference surged during the ERM period') +
    xlab('Date') +
    ylab('Rolling 12-month Std of Inflation Difference') +
    consistent_theming() + 
    ggsize(1000,600)
)
plots.append((Inflation_Difference_Volatility_Plot, "Inflation_Difference_Volatility_Plot"))

# Calculate the 12-month rolling standard deviation of the UK's industrial production growth,
# providing a measure of how volatile the production growth rate has been over time.
UK_data["IP_Growth_Volatility"] = (
    UK_data["Industrial_Production_Growth"]
    .rolling(window=12)
    .std()
)

# Plot the rolling volatility of the UK's industrial production growth, again indicating
# ERM entry and exit with dashed red lines to illustrate any changes associated with those dates.
IP_Growth_Volatility_Plot = (
    ggplot(UK_data, aes(x='Date', y='IP_Growth_Volatility')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('There was little change in volatility of IP growth during the ERM period') +
    xlab('Date') +
    ylab('Rolling 12-month Std of Inflation Difference') +  # Label indicates it's a volatility measure
    consistent_theming() + 
    ggsize(1000,600)
)
plots.append((IP_Growth_Volatility_Plot, "IP_Growth_Volatility_Plot"))

# Arrange both plots side by side in a grid for easier comparison.
gggrid([IP_Growth_Volatility_Plot, Inflation_Difference_Volatility_Plot])


### **6. Produce informative figures for the first two sections of the Black Wednesday part of the report**

In [149]:
# Filter the UK_data DataFrame for dates up to (and including) the UK's ERM entry date,
# then plot the CPI inflation trend in this period. This highlights how inflation behaved
# leading up to their ERM membership.
UK_inflation_plot_before_ERM = (
    ggplot(UK_data[UK_data['Date'] <= date_joined_ERM], aes(x='Date', y='CPI_Inflation')) +
    geom_line(color='blue', size=1.2) +
    ggtitle("UK Inflation was well above 2% during the years preceding the UK's entrance to the ERM") +
    xlab('Date') +
    ylab('Inflation (%)') +
    consistent_theming() + 
    ggsize(1000,600)
)

# Append the plot to the list for later usage (e.g., display or saving)
plots.append((UK_inflation_plot_before_ERM, "UK_inflation_plot_before_ERM"))

# Display the plot object directly
UK_inflation_plot_before_ERM

## **7. Perform statistical tests/analysis to inform the comprehension**

In [ ]:
def assign_periods(df, date_col, date_joined, date_left):
    # Define boolean conditions that check whether each date in the DataFrame
    # is before, during, or after the ERM period
    conditions = [
        df[date_col] < date_joined,
        (df[date_col] >= date_joined) & (df[date_col] <= date_left),
        df[date_col] > date_left
    ]
    
    # Label each condition with the corresponding period name
    choices = ['Pre-ERM', 'During-ERM', 'Post-ERM']
    
    # Assign a 'Period' column to the DataFrame based on the conditions above.
    # If a record doesn't match any condition, assign 'Unknown' by default.
    df['Period'] = np.select(conditions, choices, default='Unknown')
    
    # Return the modified DataFrame
    return df

def run_tests(df, measure):
    # Filter the data for each relevant period and drop missing values
    pre_data = df.loc[df['Period'] == 'Pre-ERM', measure].dropna()
    dur_data = df.loc[df['Period'] == 'During-ERM', measure].dropna()
    post_data = df.loc[df['Period'] == 'Post-ERM', measure].dropna()
    
    # Perform Welch's two-sample t-tests (which do not assume equal variances):
    # 1) Pre-ERM vs During-ERM
    # 2) During-ERM vs Post-ERM
    pre_dur_ttest  = ttest_ind(pre_data, dur_data, equal_var=False)
    dur_post_ttest = ttest_ind(dur_data, post_data, equal_var=False)
    
    # Prepare data for the Wilcoxon signed-rank test, which is typically used for
    # paired samples. We truncate both series to the same length before testing.
    pre_dur_len  = min(len(pre_data), len(dur_data))
    dur_post_len = min(len(dur_data), len(post_data))
    
    # Perform Wilcoxon signed-rank tests:
    # 1) Pre-ERM vs During-ERM (paired over the truncated length)
    # 2) During-ERM vs Post-ERM (paired over the truncated length)
    pre_dur_wilcoxon  = wilcoxon(pre_data.iloc[:pre_dur_len], dur_data.iloc[:pre_dur_len])
    dur_post_wilcoxon = wilcoxon(dur_data.iloc[:dur_post_len], post_data.iloc[:dur_post_len])
    
    # Return a dictionary of test descriptions and their results
    return {
        't-test (Pre vs During)': pre_dur_ttest,
        't-test (During vs Post)': dur_post_ttest,
        'Wilcoxon (Pre vs During)': pre_dur_wilcoxon,
        'Wilcoxon (During vs Post)': dur_post_wilcoxon
    }

# -----------------------------
# 1. Assign Periods
# -----------------------------
# Categorise each date in UK_data into Pre-ERM, During-ERM, or Post-ERM.
UK_data = assign_periods(UK_data, 'Date', date_joined_ERM, date_left_ERM)

# Do the same for Swedish_data using the same ERM dates.
# -----------------------------
# 2. Run Volatility Tests (UK)
# -----------------------------
# Run t-tests and Wilcoxon tests on 'Volatility' for each period in UK_data.
vol_results = run_tests(UK_data, 'Volatility')

# Print out the results for real exchange rate volatility in a readable format.
print("UK Real Excahnge Rate Volatility Tests")
for desc, result in vol_results.items():
    print(f"{desc}: {result}")

# -----------------------------
# 3. Run IP Growth Volatility Tests (UK)
# -----------------------------
# Run the same statistical tests on 'IP_Growth_Volatility' to see if
# industrial production volatility changed significantly across periods.
ip_results = run_tests(UK_data, 'IP_Growth_Volatility')

print("\nUK IP Growth Volatility Tests")
for desc, result in ip_results.items():
    print(f"{desc}: {result}")

# -----------------------------
# 4. Inflation Differential
# -----------------------------
# Reassign periods for German_UK_inflation to use consistent date range labels
df_infl = assign_periods(German_UK_inflation, 'Date', date_joined_ERM, date_left_ERM)

# Run tests on the volatility of the inflation difference (UK - Germany),
# to check whether it changed significantly before, during, or after the ERM period.
infl_results = run_tests(df_infl, 'Inflation_Difference_Volatility')

print("\nInflation Differential (Germany - UK) Tests")
for desc, result in infl_results.items():
    print(f"{desc}: {result}")


UK Real Excahnge Rate Volatility Tests
t-test (Pre vs During): TtestResult(statistic=np.float64(3.79753612700855), pvalue=np.float64(0.0006208284650843367), df=np.float64(31.79655425544075))
t-test (During vs Post): TtestResult(statistic=np.float64(-1.1863748400529912), pvalue=np.float64(0.2431792844191029), df=np.float64(36.30991625771277))
Wilcoxon (Pre vs During): WilcoxonResult(statistic=np.float64(39.0), pvalue=np.float64(0.0008462667465209961))
Wilcoxon (During vs Post): WilcoxonResult(statistic=np.float64(27.0), pvalue=np.float64(0.00014960765838623047))

UK IP Growth Volatility Tests
t-test (Pre vs During): TtestResult(statistic=np.float64(0.8998944749297078), pvalue=np.float64(0.3708633807732262), df=np.float64(80.41532674277784))
t-test (During vs Post): TtestResult(statistic=np.float64(-1.954328133834658), pvalue=np.float64(0.05631745243409211), df=np.float64(49.55120357046914))
Wilcoxon (Pre vs During): WilcoxonResult(statistic=np.float64(80.0), pvalue=np.float64(0.04567968

In [151]:
# Save all the plots to a folder
for _plot, _plot_name in plots:
    ggsave(plot=_plot, filename=f"{_plot_name}.png", path="../figures")

In [152]:
# Define a function that performs a simple linear regression using OLS (Ordinary Least Squares),
# regressing a target variable 'var' on 'Volatility'.
def regression(regression_data, var):
    # Extract 'Volatility' as the explanatory variable (X) and the specified column ('var') as the dependent variable (y).
    X = regression_data['Volatility']
    y = regression_data[var]

    # Insert a constant term into the model for the intercept.
    X = sm.add_constant(X)
    
    # Return a summary of the fitted regression model.
    return sm.OLS(y, X).fit().summary()

# Build a DataFrame containing the UK's Real Exchange Rate Volatility and Industrial Production Growth Volatility,
# ensuring rows with missing values are excluded.  
regression_data = pd.DataFrame({
    'Volatility': UK_data['Volatility'],
    'IP_Growth_Volatility': UK_data['IP_Growth_Volatility']
}).dropna()

# Replace any infinite values with NaN, then drop rows with NaNs again to clean the dataset further.
regression_data = regression_data.replace([np.inf, -np.inf], np.nan).dropna()

# Run the regression on how 'Volatility' (explanatory) might explain 'IP_Growth_Volatility' (dependent).
print(regression(regression_data, 'IP_Growth_Volatility'))

# Merge UK data and Germany-UK inflation data on 'Date' to combine relevant columns from both datasets.
merged_data = pd.merge(UK_data, German_UK_inflation, on='Date', how='inner')

# Build a DataFrame containing the UK's Real Exchange Rate Volatility and the Inflation Difference Volatility
# (UK - Germany), dropping missing rows for a clean dataset.
regression_data = pd.DataFrame({
    'Volatility': merged_data['Volatility'],
    'Inflation_Difference_Volatility': merged_data['Inflation_Difference_Volatility']
}).dropna()

# Run the regression on how 'Volatility' (explanatory) might explain 'Inflation_Difference_Volatility' (dependent).
print(regression(regression_data, 'Inflation_Difference_Volatility'))


                             OLS Regression Results                             
Dep. Variable:     IP_Growth_Volatility   R-squared:                       0.043
Model:                              OLS   Adj. R-squared:                  0.039
Method:                   Least Squares   F-statistic:                     10.70
Date:                  Sun, 23 Mar 2025   Prob (F-statistic):            0.00123
Time:                          19:19:21   Log-Likelihood:                -269.98
No. Observations:                   241   AIC:                             544.0
Df Residuals:                       239   BIC:                             550.9
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4192      0.093

In [153]:


# 1. Create period classifications
UK_data['Period_Class'] = pd.cut(
    UK_data['Date'], 
    bins=[pd.Timestamp('1987-01-01'), date_joined_ERM, date_left_ERM, pd.Timestamp('1996-01-01')],
    labels=['Pre-ERM', 'During-ERM', 'Post-ERM']
)

German_UK_inflation['Period_Class'] = pd.cut(
    German_UK_inflation['Date'], 
    bins=[pd.Timestamp('1987-01-01'), date_joined_ERM, date_left_ERM, pd.Timestamp('1996-01-01')],
    labels=['Pre-ERM', 'During-ERM', 'Post-ERM']
)

# 2. Calculate means by period - address FutureWarning
uk_period_means = UK_data.groupby('Period_Class', observed=False)[
    ['Real_Effective_Exchange_Rate', 'Industrial_Production_Growth']
].mean()

inf_period_means = German_UK_inflation.groupby('Period_Class', observed=False)[
    ['Difference']
].mean()

# 3. Calculate percent changes after leaving ERM
pct_change_reer = ((uk_period_means.loc['Post-ERM', 'Real_Effective_Exchange_Rate'] - 
                   uk_period_means.loc['During-ERM', 'Real_Effective_Exchange_Rate']) / 
                   uk_period_means.loc['During-ERM', 'Real_Effective_Exchange_Rate']) * 100

pct_change_ip = ((uk_period_means.loc['Post-ERM', 'Industrial_Production_Growth'] - 
                 uk_period_means.loc['During-ERM', 'Industrial_Production_Growth']) / 
                 uk_period_means.loc['During-ERM', 'Industrial_Production_Growth']) * 100

pct_change_inf = ((inf_period_means.loc['Post-ERM', 'Difference'] - 
                  inf_period_means.loc['During-ERM', 'Difference']) / 
                  inf_period_means.loc['During-ERM', 'Difference']) * 100

# 4. Perform t-tests to assess statistical significance
# Real Exchange Rate
t_stat_reer, p_val_reer = ttest_ind(
    UK_data[UK_data['Period_Class'] == 'During-ERM']['Real_Effective_Exchange_Rate'],
    UK_data[UK_data['Period_Class'] == 'Post-ERM']['Real_Effective_Exchange_Rate']
)

# Inflation Difference
t_stat_inf, p_val_inf = ttest_ind(
    German_UK_inflation[German_UK_inflation['Period_Class'] == 'During-ERM']['Difference'],
    German_UK_inflation[German_UK_inflation['Period_Class'] == 'Post-ERM']['Difference']
)

# Industrial Production Growth
t_stat_ip, p_val_ip = ttest_ind(
    UK_data[UK_data['Period_Class'] == 'During-ERM']['Industrial_Production_Growth'],
    UK_data[UK_data['Period_Class'] == 'Post-ERM']['Industrial_Production_Growth']
)

# 5. Create dummy variables for regression analysis
UK_data['During_ERM'] = (UK_data['Period_Class'] == 'During-ERM').astype(int)
UK_data['Post_ERM'] = (UK_data['Period_Class'] == 'Post-ERM').astype(int)
German_UK_inflation['During_ERM'] = (German_UK_inflation['Period_Class'] == 'During-ERM').astype(int)
German_UK_inflation['Post_ERM'] = (German_UK_inflation['Period_Class'] == 'Post-ERM').astype(int)

# 6. Run regressions to assess impact of ERM periods
model_reer = smf.ols('Real_Effective_Exchange_Rate ~ During_ERM + Post_ERM', data=UK_data).fit()
model_ip = smf.ols('Industrial_Production_Growth ~ During_ERM + Post_ERM', data=UK_data).fit()
model_inf = smf.ols('Difference ~ During_ERM + Post_ERM', data=German_UK_inflation).fit()

# 7. Black Wednesday specific analysis - 3 months before/after
bw_start = date_left_ERM - pd.DateOffset(months=3)
bw_end = date_left_ERM + pd.DateOffset(months=3)

# Filter data for Black Wednesday period - address SettingWithCopyWarning
bw_uk_data = UK_data[(UK_data['Date'] >= bw_start) & (UK_data['Date'] <= bw_end)].copy()
bw_inf_data = German_UK_inflation[(German_UK_inflation['Date'] >= bw_start) & 
                                 (German_UK_inflation['Date'] <= bw_end)].copy()

# Create before/after indicators - using .loc to avoid warnings
bw_uk_data.loc[:, 'After_BW'] = (bw_uk_data['Date'] >= date_left_ERM).astype(int)
bw_inf_data.loc[:, 'After_BW'] = (bw_inf_data['Date'] >= date_left_ERM).astype(int)

# Calculate immediate impact
bw_impact_reer = bw_uk_data.groupby('After_BW', observed=False)['Real_Effective_Exchange_Rate'].mean()
bw_impact_ip = bw_uk_data.groupby('After_BW', observed=False)['Industrial_Production_Growth'].mean()
bw_impact_inf = bw_inf_data.groupby('After_BW', observed=False)['Difference'].mean()

# 8. Create results summary
results = pd.DataFrame({
    'Variable': ['Real Exchange Rate', 'Inflation Difference', 'Industrial Production Growth'],
    'During ERM Mean': [uk_period_means.loc['During-ERM', 'Real_Effective_Exchange_Rate'], 
                       inf_period_means.loc['During-ERM', 'Difference'],
                       uk_period_means.loc['During-ERM', 'Industrial_Production_Growth']],
    'Post ERM Mean': [uk_period_means.loc['Post-ERM', 'Real_Effective_Exchange_Rate'], 
                     inf_period_means.loc['Post-ERM', 'Difference'],
                     uk_period_means.loc['Post-ERM', 'Industrial_Production_Growth']],
    'Pct Change After ERM (%)': [pct_change_reer, pct_change_inf, pct_change_ip],
    'p-value (t-test)': [p_val_reer, p_val_inf, p_val_ip],
    'Immediate BW Impact (%)': [
        ((bw_impact_reer[1] - bw_impact_reer[0])/bw_impact_reer[0])*100,
        ((bw_impact_inf[1] - bw_impact_inf[0])/bw_impact_inf[0])*100,
        ((bw_impact_ip[1] - bw_impact_ip[0])/bw_impact_ip[0])*100
    ]
})

results

,Variable,During ERM Mean,Post ERM Mean,Pct Change After ERM (%),p-value (t-test),Immediate BW Impact (%)
0,Real Exchange Rate,123.932200,107.134809,-13.553693,3.047811e-35,-11.430667
1,Inflation Difference,2.112330,-0.383985,-118.178275,5.813904e-06,-65.452857
2,Industrial Production Growth,-1.848801,2.993559,-261.918939,1.380592e-08,71.875957
